In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.cluster import DBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD

In [2]:
source_data = pd.read_csv('view_food_clean.csv')

/var/folders/1p/gl48js3964n3jh2chpp0xk9c0000gn/T/ipykernel_23978/2772101045.py:1: DtypeWarning: Columns (36,52) have mixed types. Specify dtype option on import or set low_memory=False.
  source_data = pd.read_csv('view_food_clean.csv')


In [3]:
source_data_copy = source_data.copy()

In [4]:
source_data_copy = source_data_copy[source_data_copy['merged_to'].isna()]

In [ ]:
source_data_copy

In [6]:
cols_to_concat = [
    'name', 'name_search', 'energy', 'protein', 'fat',
    'saturated_fatty_acid', 'carbohydrates', 'remarks_carbohydrates',
    'synonyms', 'brands', 'brands_search', 'bron', 'categories'
]

# 1. Handle missing values: replace with an empty string ('')
# 2. Convert to string: convert all values in the selected columns to string type
source_data_copy[cols_to_concat] = source_data_copy[cols_to_concat].fillna('').astype(str)

# 3. Concatenate the string values, separated by a space
# The .agg(' '.join, axis=1) method concatenates all string values in a row with a space in between
source_data_copy['concatenated_values'] = source_data_copy[cols_to_concat].agg(' '.join, axis=1)

In [7]:
# settings that you use for count vectorizer will go here 
tfidf_vectorizer = TfidfVectorizer(use_idf=True) 

# just send in all your docs here 
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(source_data_copy['concatenated_values'])

# get the first vector out (for the first document) 
first_vector_tfidfvectorizer = tfidf_vectorizer_vectors[0] 

In [8]:
dbscan = DBSCAN(eps=0.3, min_samples=3, metric='cosine') # Using cosine distance for better text vector comparison

# Fit DBSCAN on the TF-IDF matrix (one row per product) and save labels to product_text
labels = dbscan.fit_predict(tfidf_vectorizer_vectors)
source_data_copy['cluster_id'] = labels

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
nonly_noise = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters)
print("Estimated number of noise points: %d" % nonly_noise)

Estimated number of clusters: 651
Estimated number of noise points: 12954


In [1]:
source_data_copy

NameError: name 'source_data_copy' is not defined